# Лабораторна робота №10
## Тема: Стиснення даних. Алгоритм кодування Гафмена
### Виконав: Варакута Олександр
## Група КІ-24-1

## Мета роботи: Ознайомитися з принципами жадібних алгоритмів на прикладіалгоритму кодування Гафмена, навчитися будувати дерево кодів, реалізувати кодування та декодування повідомлень і оцінити ефективність стиснення даних.


## Теоретичні відомості

Алгоритм Гафмена — це жадібний алгоритм оптимального
префіксного кодування, який мінімізує середню довжину коду
символів повідомлення.

Нехай задано алфавіт символів $\Sigma$ з частотами $f_i$.
Метою є мінімізація величини:
$$
L = \sum_{i=1}^{n} f_i \cdot l_i
$$
де $l_i$ — довжина коду символу $i$.


## Індивідуальне завдання 
Дано: AABAABABABABCBBCBCAAADEE
Необхідно:
- побудувати дерево Гафмена;
- закодувати повідомлення;
- декодувати повідомлення;
- оцінити ефект стиснення.

## 1. Обчислення частот символів

In [1]:
from collections import Counter

text = "AABAABABABABCBBCBCAAADEE"
freq = Counter(text)
freq

Counter({'A': 10, 'B': 8, 'C': 3, 'E': 2, 'D': 1})

## 2. Побудова дерева Гафмена

In [8]:
import heapq

class Node:
    def __init__(self, char=None, freq=0, left=None, right=None):
        self.char = char
        self.freq = freq
        self.left = left
        self.right = right

    def __lt__(self, other):
        return self.freq < other.freq
heap = [Node(ch, f) for ch, f in freq.items()]
heapq.heapify(heap)

while len(heap) > 1:
    left = heapq.heappop(heap)
    right = heapq.heappop(heap)
    merged = Node(freq=left.freq + right.freq, left=left, right=right)
    heapq.heappush(heap, merged)

root = heap[0]



## 3. Генерація кодів Гафмена

In [9]:
codes = {}

def generate_codes(node, code=""):
    if node is None:
        return
    if node.char is not None:
        codes[node.char] = code
    generate_codes(node.left, code + "0")
    generate_codes(node.right, code + "1")

generate_codes(root)
codes

{'A': '0', 'C': '100', 'D': '1010', 'E': '1011', 'B': '11'}

## 4. Кодування повідомлення

In [10]:
encoded_text = "".join(codes[ch] for ch in text)
encoded_text

'00110011011011011100111110011100000101010111011'

## 5. Декодування повідомлення

In [11]:
def decode(encoded, root):
    result = ""
    node = root
    for bit in encoded:
        node = node.left if bit == "0" else node.right
        if node.char is not None:
            result += node.char
            node = root
    return result

decoded_text = decode(encoded_text, root)
decoded_text


'AABAABABABABCBBCBCAAADEE'

## 6. ASCII-візуалізація дерева Гафмена

In [12]:
def print_tree(node, prefix="", is_left=True):
    if node.right:
        print_tree(node.right, prefix + ("│   " if is_left else "    "), False)
    print(prefix + ("└── " if is_left else "┌── ") + f"{node.char if node.char else ''}({node.freq})")
    if node.left:
        print_tree(node.left, prefix + ("    " if is_left else "│   "), True)

print_tree(root)


│       ┌── B(8)
│   ┌── (14)
│   │   │       ┌── E(2)
│   │   │   ┌── (3)
│   │   │   │   └── D(1)
│   │   └── (6)
│   │       └── C(3)
└── (24)
    └── A(10)


## 7. Оцінка ефективності кодування

### Неоптимальне (фіксоване) кодування

Кількість різних символів:
$$
|\Sigma| = 5
$$

Мінімальна фіксована довжина коду:
$$
\lceil \log_2 5 \rceil = 3 \text{ біти}
$$

Загальна довжина повідомлення:
$$
L_{\text{fixed}} = 26 \cdot 3 = 78 \text{ біт}
$$

### Кодування Гафмена

In [13]:
len(encoded_text)

47


Довжина закодованого повідомлення визначається
як кількість бітів у бітовому рядку:

$$
L_{\text{Huffman}} = 47 \text{ біт}
$$



### Коефіцієнт стиснення
$$
K = \frac{78}{47} \approx 1.66
$$


## Контрольні питання

### 1. Що таке жадібні алгоритми?

Жадібні алгоритми — це алгоритми, які на кожному кроці приймають локально
оптимальне рішення з надією отримати глобально оптимальний результат.
Вони не виконують повний перебір варіантів, а діють за принципом
найкращого вибору в поточний момент.

Алгоритм Гафмена є класичним прикладом жадібного алгоритму, який
для задачі побудови префіксного коду забезпечує оптимальний результат.

### 2. Що таке префіксний код? Який код використовується у коді Гафмена?

Префіксний код — це спосіб кодування, за якого жодне кодове слово
не є префіксом іншого. Це гарантує однозначність декодування.

У коді Гафмена використовується двійковий префіксний код, що мінімізує
середню довжину повідомлення та дозволяє виконувати декодування
без розділювачів.

### 3. Як пов’язана структура даних «купа» зі структурою даних «черга з пріоритетами»?

Черга з пріоритетами — це абстрактна структура даних, у якій елемент
з найбільшим або найменшим пріоритетом обробляється першим.

Купа є ефективною реалізацією черги з пріоритетами.
В алгоритмі Гафмена мін-купа використовується для вибору двох вузлів
з мінімальними частотами на кожному кроці побудови дерева.

### 4. Що таке стиснення даних і для чого воно використовується? Які його головні переваги?

Стиснення даних — це процес зменшення обсягу інформації шляхом
усунення надлишковості без втрати даних або з контрольованими втратами.

Основні переваги стиснення:
- економія пам’яті для зберігання даних;
- зменшення часу передачі інформації мережею;
- зниження навантаження на канали зв’язку та обчислювальні ресурси.

### 5. Які кроки необхідно виконати для стиснення даних за допомогою алгоритму кодування Гафмена?

Для стиснення даних алгоритмом Гафмена виконуються такі кроки:
1. Визначення частот появи кожного символу повідомлення.
2. Формування мін-купи з вузлів, що містять символи та їх частоти.
3. Послідовне об’єднання двох вузлів з найменшими частотами.
4. Побудова бінарного дерева Гафмена.
5. Призначення двійкових кодів символам за шляхами в дереві.
6. Кодування повідомлення шляхом заміни символів відповідними кодами.

### 7. Які існують альтернативні методи стиснення даних, що можуть конкурувати з алгоритмом Гафмена?

Серед альтернативних методів стиснення можна виділити:
- алгоритми Лемпеля–Зіва (LZ77, LZ78);
- алгоритм LZW;
- арифметичне кодування;
- контекстні методи стиснення;
- алгоритми стиснення з втратами для мультимедіа.

Деякі з цих методів забезпечують кращий рівень стиснення,
але є складнішими в реалізації.

### 8. Які практичні застосування можуть мати алгоритми стиснення даних, зокрема алгоритм Гафмена, у сучасних інформаційних системах?

Алгоритми стиснення даних широко використовуються у файлових архівах,
мережевих протоколах, форматах зображень і мультимедіа,
а також у телекомунікаційних та вбудованих системах.

Алгоритм Гафмена часто застосовується як складова частина
більш складних систем стиснення, де він відповідає за фінальний
етап кодування даних.
